## Self-practice Task

Train a CNN model and predict the categories in `Caltech 256` dataset:
1. Load the data from `torchvision.datasets.Caltech256`
1. Split the data to train, validation and test
1. Define a CNN model with achitecture of your choice
1. Train the model and log the loss and accuracy at every epoch (on train, validation and test set)
1. Use a pretrained (such as `VGG16`) model for the same task and compare the models number of parameters together with accuracy

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import Caltech256
from torchvision.models import vgg16
from torch import nn
import torch.optim as optim
from glob import glob
from PIL import Image
import random
from matplotlib import pyplot as plt
import numpy as np
import torchvision
from torchvision.transforms import Resize

dataset = Caltech256(root='./', download=True)
dataset.download()


Files already downloaded and verified
Files already downloaded and verified


In [2]:
torch.cuda.memory_reserved(0)

0

In [3]:
image_list = glob('./caltech256/*/*/*.jpg')

class ImageDataset(Dataset):
    def __init__(self, image_list, end_size=(100,100)):
        self.image_lsit = image_list
        self.end_size = end_size

    def __len__(self):
        return (len(self.image_lsit))

    def __getitem__(self, i):
        image = plt.imread(self.image_lsit[i])
        image = Image.fromarray(image).convert('RGB')
        image = np.asarray(image).astype(np.float) / 255
        image = torch.tensor(image, dtype=torch.float).permute([2,0,1])
        image = Resize(size=self.end_size)(image)
        label = torch.tensor(int(image_list[i].split('\\')[3][:3]), dtype=torch.uint8)

        return image, label

batch_size = 16

random.shuffle(image_list)
train_val_test_size = [0.6,0.2,0.2]
x_train = image_list[:int(len(image_list)*train_val_test_size[0])]
x_val = image_list[len(x_train):len(x_train)+int(len(image_list)*train_val_test_size[1])]
x_test = image_list[-int(len(image_list)*train_val_test_size[2]):]

x_train = ImageDataset(x_train)
x_test = ImageDataset(x_test)
x_val = ImageDataset(x_val)

x_train = DataLoader(dataset=x_train, batch_size=batch_size, shuffle=False, pin_memory_device='cpu')
x_test = DataLoader(dataset=x_test, batch_size=batch_size, shuffle=False, pin_memory_device='cpu')
x_val = DataLoader(dataset=x_val, batch_size=batch_size, shuffle=False, pin_memory_device='cpu')

torch.cuda.memory_reserved(0)

0

In [4]:
class CNN_1(nn.Module):
    # Convolution formula: ((n + 2p - f) / s) + 1

    def __init__(self):
        super(CNN_1, self).__init__()
        self.layers = []
        self.layers.append(nn.Conv2d(3, 10, kernel_size=5))
        self.layers.append(nn.MaxPool2d(2))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(10, 20, kernel_size=5))
        self.layers.append(nn.Dropout2d())
        self.layers.append(nn.MaxPool2d(2))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Flatten(1))
        self.layers.append(nn.Linear(9680, 2048)) # calculated manually
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Dropout(p=0.5))
        self.layers.append(nn.Linear(2048, 256))
        self.layers.append(nn.LogSoftmax(1))

        for i, n in enumerate(self.layers):
            self.add_module(f'layer_{i}' ,n)

    def forward(self, x):
        # Exercise: calclulate shape after each layer
        for n in self.children():
            #print(n)
            x = n(x)
        #print(x)
        return x

In [5]:
def train(model, device, train_loader, optimizer, epoch, log_interval=700):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = torch.nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += torch.nn.functional.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
lr = 0.001
momentum=0.5
log_interval = 400
device = 'cpu'
epochs = 3

model_s = CNN_1().to(device)
optimizer = optim.SGD(model_s.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_s, device, x_train, optimizer, epoch, log_interval)
    test(model_s, device, x_val)
    torch.save(model_s.state_dict(), "My_c256.pt")

torch.cuda.empty_cache()

/home/thornail/anaconda3/envs/DS_1/lib/python3.9/site-packages/torch/utils/data/dataloader.py:645: UserWarning: pin memory device is set and pin_memory flag is not used then device pinned memory won't be usedplease set pin_memory to true, if you need to use the device pin memory
  warnings.warn(warn_msg)
/tmp/ipykernel_18686/31108544.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  image = np.asarray(image).astype(np.float) / 255


Train Epoch: 1 [0/18364 (0%)]	Loss: 5.540532
Train Epoch: 1 [6400/18364 (35%)]	Loss: 5.535106
Train Epoch: 1 [12800/18364 (70%)]	Loss: 5.597634


In [ ]:
model = CNN_1().to(device)

for i, n in enumerate(x_train):
    data = n[0].to(device)
    print(data.shape)
    print(model.forward(data).shape)
    if i == 0:
        break

In [ ]:
model_vgg = vgg16(pretrained=True).cuda()
model_vgg.add_module("End ReLu", nn.ReLU())
model_vgg.add_module("End linear", nn.Linear(1000, 256))
model_vgg.add_module("Softmax", nn.LogSoftmax(1))
for n in model_vgg.children():
    print(n)

In [ ]:
lr = 0.000001
momentum=0.5
log_interval = 300
device = 'cpu'
epochs = 2

optimizer = optim.SGD(model_vgg.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_vgg, device, x_train, optimizer, epoch, log_interval)
    test(model_vgg, device, x_val)
    torch.save(model_s.state_dict(), "vgg16.pt")